In [12]:
import os
import math
from azureml.widgets import RunDetails
from azureml.train.hyperdrive import *
from azureml.train.estimator import Estimator
from azureml.core import Workspace, Datastore, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

## Initialize workspace

To access an Azure ML Workspace, you will need to import the AML library and the following information:
* A name for your workspace (in our example - `hal`)
* Your subscription id (can be obtained by running `az account list`)
* The resource group name (in our case `robots`)

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace/?WT.mc_id=absa-notebook-abornst) object from the existing workspace you created in the Prerequisites step or create a new one. 

In [3]:
#subscription_id = ''
#resource_group  = 'absa'
#workspace_name  = 'absa_space'
#ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#ws.write_config()

try:
    ws = Workspace.from_config()
    print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')
    print('Library configuration succeeded')
except:
    print('Workspace not found')

hal	westus2	robots	westus2
Library configuration succeeded


## Compute

There are two computer option run once(preview) and persistent compute for this demo we will use persistent compute to learn more about run once compute check out the [docs](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute?WT.mc_id=absa-notebook-abornst).

In [4]:
# Choose a name for your CPU cluster
cluster_name = "gandalf"

# Verify that cluster does not exist already
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2',
                                                           vm_priority='lowpriority',
                                                           min_nodes=1,
                                                           max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.


### Get Datastore Reference

In [5]:
ds = Datastore.get(ws, 'absa')

## Fine-Tuning NLP Archictect  with AzureML HyperDrive
Although ABSA is an unsupervised method it's hyper parameters such as the aspect and opinion word thresholds can be fined tuned if provided with a small sample of labeled data

In [6]:
param_sampling = RandomParameterSampling({
         '--asp_thresh': choice(range(2,5)),
         '--op_thresh': choice(range(2,5)), 
         '--max_iter': choice(range(2,5))
    })

### Early Termination Policy
First we will define an early terminination policy. [Median stopping](https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.medianstoppingpolicy?WT.mc_id=absa-notebook-abornst) is an early termination policy based on running averages of primary metrics reported by the runs. This policy computes running averages across all training runs and terminates runs whose performance is worse than the median of the running averages. 

This policy takes the following configuration parameters:

- evaluation_interval: the frequency for applying the policy (optional parameter).
- delay_evaluation: delays the first policy evaluation for a specified number of intervals (optional parameter).


In [7]:
early_termination_policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=0)

Refer [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters#specify-early-termination-policy?WT.mc_id=absa-notebook-abornst) for more information on the Median stopping policy and other policies available.

Early termination policy is effective when your training script reports primary metric periodically, several times during the execution. In this case, when the metric is not getting better, the script can be terminated. In our case, since we report the metric only at the end of the execution, early termination is not effective.

Finally, we define our Hyper Drive configuration to maximize our Model's weighted F1 score. Hyper Drive can optimize any metric can be optimized as long as it's logged by the training script. 


In [8]:
nlp_est = Estimator(source_directory='.',
                   compute_target=cluster,
                   environment_variables = {'NLP_ARCHITECT_BE':'CPU'},
                   entry_script='train.py',
                   pip_packages=['git+https://github.com/NervanaSystems/nlp-architect.git@absa',
                                 'spacy==2.1.8']
)

hd_config = HyperDriveConfig(estimator=nlp_est,
                            hyperparameter_sampling=param_sampling,
                            #policy=early_termination_policy,
                            primary_metric_name='accuracy',
                            primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                            max_total_runs=16,
                            max_concurrent_runs=4)

Finally, lauch the hyperparameter tuning job.

In [9]:
experiment = Experiment(workspace=ws, name='absa_hyperdrive')

In [10]:
hyperdrive_run = experiment.submit(hd_config)

In [11]:
hyperdrive_run.id

'absa_hyperdrive_1579824130849136'

In [13]:
hyperdrive_run = [r for r in experiment.get_runs() if r.id == 'absa_hyperdrive_1579824130849136'][0]

### Monitor HyperDrive runs
We can monitor the progress of the runs with the following Jupyter widget. 

In [14]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
hyperdrive_run.cancel()

### Find the best model
Once all the runs complete, we can find the run that produced the model with the highest evaluation (METRIC TBD).

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run)
print('Best Run is:\n  F1: {0:.5f}'.format(
        best_run_metrics['f1_weighted']
     ))